# Inquinamento Airnet

In [62]:
import json
from io import StringIO

import requests
from sseclient import SSEClient
from tqdm import tqdm
import geopandas as gpd

In [63]:
# RAW
PATH_INQUINAMENTO_INGESTION_RAW = "../Data/Raw/Inquinamento/Airnet/inquinamento-airnet_media_raw.geojson"

# CLEAN
PATH_INQUINAMENTO_INGESTION_CLEAN = "../Data/Clean/Inquinamento/Airnet/inquinamento-airnet_ingestion_clean.geojson"

## Scraping

Otteniamo gli id delle stazioni di Milano

In [64]:
URL = "https://airnet.waqi.info/airnet/validator/check/112273" # Una stazione a caso di milano\
risposta = requests.get(URL)
risposta_json = risposta.json()
neighbors = risposta_json["data"]["neighbors"]  # Da questa riusciamo a ricavare tutte le stazioni "vicine"

inizializziamo il geojson con id e coordinate delle stazioni

In [65]:
geojson = {
    "type": "FeatureCollection",
    "name": "Inquinamento",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:EPSG::6707"
        }
    },
    "features": []
}
for geo, id in zip(neighbors["geos"], neighbors["ids"]):
    if not id == 353767: # Stazione marco emme, Milan, Italy che non contiene dati pm10 e pm25
        geojson["features"].append(
            {
                "type": "Feature",
                "properties": {
                    "id": id,
                    "longitudine": geo[1],
                    "latitudine": geo[0],
                    "name": "",
                    "pm10": [],
                    "pm25": []
                },
                "geometry": {
                    "type": "Point",
                    "coordinates": [geo[1], geo[0]]
                }
            }
        )

Estraiamo i dati di ogni stazione (per i 2 inquinanti) e li inseriamo direttamente tra le "features" >> "properties" del geojson inizializzato

In [66]:
pbar = tqdm(total=len(geojson["features"]), desc="Scraping progress...")

for feature in geojson["features"]:
    for inquinante in ["pm10", "pm25"]:
        url = f"https://airnet.waqi.info/airnet/sse/historic/daily/{feature["properties"]["id"]}?specie={inquinante}"
        response = requests.get(url, params={"specie": inquinante}, headers={"Accept": "text/event-stream"}, stream=True)
        client = SSEClient(response)
        for i, event in enumerate(client.events()):
            if event.event != "error":
                try:
                    data = json.loads(event.data)
                    if i == 0:
                        feature["properties"]["name"] = data["meta"]["name"]
                    elif isinstance(data, dict): #Ogni tanto ci sono righe sporche di stringhe
                        feature["properties"][inquinante].append(data)
                except json.JSONDecodeError:
                    print("Errore JSON:", event.data)
    pbar.update(1)

salvo il file geojson creato in Raw

In [67]:
with open(PATH_INQUINAMENTO_INGESTION_RAW, 'w') as file:
    json.dump(geojson, file, indent=4)

## ETL

A partire dai dati giornalieri dei 2 inquinanti (pm10 e pm25) ricaviamo gli aggregati per anno sottoforma di media delle mediane giornaliere.

In [68]:
def set_mean_pm(row):
    for pm in ["pm10", "pm25"]:
        df = gpd.pd.read_json(StringIO(row[pm]))
        df["day"] = gpd.pd.to_datetime(df["day"])
        df = df[df['day'].dt.year.isin([2023])].reset_index()
        if len(df) > 200:
            row[pm] = df["median"].mean()
        else:
            row[pm] = None
    return row

gdf = gpd.read_file(PATH_INQUINAMENTO_INGESTION_RAW)
gdf = gpd.GeoDataFrame(gdf.apply(set_mean_pm, axis=1), geometry=gdf.geometry.name)

Salviamo su Clean

In [69]:
gdf.to_file(PATH_INQUINAMENTO_INGESTION_CLEAN, driver="GeoJSON")